# Deciding whether to escalate a customer support issue

For updates on the way Sagemaker or AWS behave compared to the notebook code, please refer to https://livebook.manning.com/#!/book/machine-learning-for-business/chapter-4/v-5/108

## Part 1: Load and examine the data

In [1]:
data_bucket = 'mlforbusiness'
subfolder = 'ch04'
dataset = 'inbound.csv'

In [2]:
import pandas as pd                               
import boto3
import sagemaker
import s3fs
from sklearn.model_selection import train_test_split
import json
import nltk
import csv
from time import sleep

nltk.download('punkt')

role = sagemaker.get_execution_role()
s3 = s3fs.S3FileSystem(anon=False)

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
%%time
df = pd.read_csv(f's3://{data_bucket}/{subfolder}/{dataset}')
display(df.head())

,tweet_id,author_id,created_at,in_reply_to,text,escalate
0,2,115712,Tue Oct 31 22:11:45 +0000 2017,sprintcare,@sprintcare and how do you propose we do that,False
1,3,115712,Tue Oct 31 22:08:27 +0000 2017,sprintcare,@sprintcare I have sent several private messag...,True
2,5,115712,Tue Oct 31 21:49:35 +0000 2017,sprintcare,@sprintcare I did.,False
3,16,115713,Tue Oct 31 20:00:43 +0000 2017,sprintcare,@sprintcare Since I signed up with you....Sinc...,False
4,22,115716,Tue Oct 31 22:16:48 +0000 2017,Ask_Spectrum,@Ask_Spectrum Would you like me to email you a...,False


CPU times: user 2.04 s, sys: 324 ms, total: 2.36 s
Wall time: 4.1 s


In [4]:
print(f'Number of rows in dataset: {df.shape[0]}')
print(df['escalate'].value_counts())

Number of rows in dataset: 520793
False    417800
True     102993
Name: escalate, dtype: int64


## Part 2: Get the data into the right shape

In [5]:
train_df, val_df, _, _ = train_test_split(df, df['escalate'], test_size=0.2, random_state=0)
print(f'{train_df.shape[0]} rows in training data')
print(f'{val_df.shape[0]} rows in validation data')

416634 rows in training data
104159 rows in validation data


In [6]:
%%time

def preprocess(df):
    all_rows = df.values.tolist()
    transformed_rows = list(map(transform_instance, all_rows))
    transformed_df = pd.DataFrame(transformed_rows)
    return transformed_df

def transform_instance(row):
    cur_row = []
    label = "__label__1" if row[5] == True else "__label__0" # Prefix 0 or 1 from sentiment
    cur_row.append(label)
    cur_row.extend(nltk.word_tokenize(row[4].lower()))
    return ' '.join(cur_row)

transformed_validation_rows = preprocess(val_df)
display(transformed_validation_rows.head())

,0
0,__label__1 @ 115990 no joke ... this is one of...
1,__label__0 @ amazonhelp primeira camada ... ht...
2,__label__1 @ microsofthelps my mistake
3,__label__1 @ 770932 @ americanair they notorio...
4,__label__1 @ amazonhelp neither man seems to k...


CPU times: user 25.1 s, sys: 0 ns, total: 25.1 s
Wall time: 25.1 s


In [7]:
s3_validation_data = f's3://{data_bucket}/{subfolder}/processed/validation.csv'

data = transformed_validation_rows.to_csv(
        header=False, index=False, quoting=csv.QUOTE_NONE, sep='|', escapechar='^').encode()
with s3.open(s3_validation_data, 'wb') as f:
    f.write(data)

In [8]:
%%time
transformed_train_rows = preprocess(train_df)
display(transformed_train_rows.head())

s3_train_data = f's3://{data_bucket}/{subfolder}/processed/train.csv'

data = transformed_train_rows.to_csv(
        header=False, index=False, quoting=csv.QUOTE_NONE, sep='|', escapechar='^').encode()
with s3.open(s3_train_data, 'wb') as f:
    f.write(data)

,0
0,__label__0 @ amazonhelp et en plus se faire en...
1,__label__1 @ morrisons @ 641226 standard reply...
2,__label__1 @ idea_cares @ 1936 @ 116590 this i...
3,__label__0 @ askamex yes i did weeks ago and n...
4,__label__0 @ amazonhelp i do n't want your stu...


CPU times: user 1min 41s, sys: 345 ms, total: 1min 41s
Wall time: 1min 42s


## Part 3: Create training and validation datasets

In [9]:
%%time

train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/plain', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated', 
                             content_type='text/plain', s3_data_type='S3Prefix')

CPU times: user 23 µs, sys: 0 ns, total: 23 µs
Wall time: 26.9 µs


## Part 4: Train the model

In [10]:
s3_output_location = f's3://{data_bucket}/{subfolder}/output'

sess = sagemaker.Session()

container = sagemaker.amazon.amazon_estimator.get_image_uri(
                            boto3.Session().region_name, "blazingtext", "latest")

estimator = sagemaker.estimator.Estimator(
                            container,
                            role, 
                            train_instance_count=1, 
                            train_instance_type='ml.m4.xlarge',
                            train_max_run = 600,
                            output_path=s3_output_location,
                            sagemaker_session=sess)

estimator.set_hyperparameters(
                            mode="supervised",
                            epochs=10,
                            vector_dim=10,
                            early_stopping=True,
                            patience=4,
                            min_epochs=5,
                            word_ngrams=2)

estimator.fit({'train': train_data, 'validation': validation_data})

2019-06-15 05:25:43 Starting - Starting the training job...
2019-06-15 05:25:47 Starting - Launching requested ML instances......
2019-06-15 05:26:56 Starting - Preparing the instances for training...
2019-06-15 05:27:40 Downloading - Downloading input data..
Arguments: train
[06/15/2019 05:27:57 WARNING 140114792494912] Loggers have already been setup.
[06/15/2019 05:27:57 WARNING 140114792494912] Loggers have already been setup.
[06/15/2019 05:27:57 INFO 140114792494912] nvidia-smi took: 0.0252411365509 secs to identify 0 gpus
[06/15/2019 05:27:57 INFO 140114792494912] Running single machine CPU BlazingText training using supervised mode.
[06/15/2019 05:27:57 INFO 140114792494912] Processing /opt/ml/input/data/train/train.csv . File size: 43 MB
[06/15/2019 05:27:57 INFO 140114792494912] Processing /opt/ml/input/data/validation/validation.csv . File size: 10 MB
Read 9M words
Number of words:  24996
Loading validation data from /opt/ml/input/data/validation/validation.csv
Loaded valida

## Part 5: Host the Model

In [11]:
endpoint_name = 'customer-support-ch04'
try:
    sess.delete_endpoint(endpoint_name)
    print('Warning: Existing endpoint deleted to make way for your new endpoint.')
    sleep(30)
except:
    pass

In [12]:
text_classifier = estimator.deploy(
                        initial_instance_count = 1,
                        instance_type = 'ml.m4.xlarge',
                        endpoint_name=endpoint_name)

---------------------------------------------------------------------------------------------------------------!

## Test the Model

In [13]:
tweet = "I'm not angry!"

tokenized_tweet = [' '.join(nltk.word_tokenize(tweet))]
payload = {"instances" : tokenized_tweet}
response = text_classifier.predict(json.dumps(payload))
escalate = pd.read_json(response)
escalate

,label,prob
0,[__label__0],[0.6225702762603761]


## Remove the Endpoint (optional)

Comment out this cell to remove the endpoint if you want the endpoint to exist after "run all"

In [14]:
sess.delete_endpoint(endpoint_name)